# TextCNN尝试

In [49]:
import pandas as pd
import numpy as np
import configparser
import pymysql
import matplotlib.pyplot as plt
import jieba
import jieba.analyse
import gensim
import pickle
import fasttext
from sklearn.model_selection import train_test_split
from collections import Counter
%matplotlib inline
import fool

In [6]:
config = configparser.ConfigParser()
config.read("../Data/Input/database_config/database.conf")
host = config['ASSESSMENT']['host']
user = config['ASSESSMENT']['user']
password = config['ASSESSMENT']['password']
database = config['ASSESSMENT']['database']
port = config['ASSESSMENT']['port']
charset = config['ASSESSMENT']['charset']
db = pymysql.connect(host=host, user=user, password=password, db=database, port=int(port), charset=charset)

## 一、数据预处理

### 从数据库读取标签和文本信息

In [10]:
sql_intro = "select * from company_intro_info_latest0601"
sql_tag = "select * from company_tag_info_latest0601"
data_intro = pd.read_sql(sql_intro, con=db)
data_tag = pd.read_sql(sql_tag, con=db)

In [14]:
# 剔除非概念标签和技术标签
data_ctag_only = data_tag[(data_tag.remarks != "1") & (data_tag.classify_id != 4)].copy()

In [15]:
data_ctag_only.head(1)

,id,comp_id,comp_full_name,label_id,label_name,label_refe_id,label_refe_name,classify_id,label_type,label_type_num,load_time,modify_time,load_person,src_tags,info_key,remarks,del_flag,data_source,modify_person
812677,6926269,13148896927021693000,北京微吼时代科技有限公司,None,文化娱乐行业,None,None,2,1,1,2018-06-01 19:27:15,2018-06-01 19:27:15,jhx,文化娱乐行业-视频直播,4800646622910500233,None,0,None,None


In [16]:
data_intro.head(1)

,id,comp_id,comp_full_name,intro,source_y,load_time,modify_time,load_person,modify_person,remarks,data_source,del_flag,classify_id,intro_key
0,31922,10022956169995483090,常州众股网络科技有限公司,常州众股网络科技有限公司是一家集传媒、公益、实业等领域组成的新兴集团公司，致力于让广告主的广...,smt,2018-06-01 19:27:14,2018-06-01 19:27:14,jhx,None,None,None,0,4,4781149490004686310


In [17]:
ctag_count = data_ctag_only.groupby("label_name").agg({"comp_full_name": "count"}).sort_values(by="comp_full_name", ascending=False).reset_index()

In [18]:
ctag_count[ctag_count.comp_full_name >= 1000]

,label_name,comp_full_name
0,文化创意产业,41111
1,商务服务,33067
2,新型材料产业,18280
3,科技金融,12550
4,视觉传达,11990
5,医疗健康行业,11847
6,文娱传媒,11503
7,咨询与调查,11002
8,大数据产业,9805
9,其他商务服务业,9648


### 分离出每个标签的标签链条，过滤进行中的产业链

In [19]:
def src_tags_splitter(src_tags):
    links = src_tags.split("#")
    tags_lists = [link.split("-") for link in links]
    tags = set([x for y in tags_lists for x in y])
    return tags

In [20]:
data_ctag_only["one_src_tags"] = data_ctag_only[["label_type_num", "src_tags"]].apply(lambda x: x[1].split("#")[x[0] - 1], axis=1)

In [21]:
data_ctag_only["tags_list"] = data_ctag_only.src_tags.apply(lambda x: src_tags_splitter(x))

In [22]:
filter_list = "泛娱乐,生物科学,科技金融,智能网联汽车,科技物流,文化娱乐行业,数字媒体行业,广告营销行业,游戏行业,教育培训行业,电子商务行业,汽车交通,房产家装,医疗健康行业,新零售".split(",")

In [23]:
data_ctag_filtered = data_ctag_only[~data_ctag_only.one_src_tags.apply(lambda x: x.split("-")[0]).isin(filter_list)][["comp_id", "comp_full_name", "label_name", "src_tags", "one_src_tags", "tags_list"]].copy()

In [24]:
data_ctag_filtered.head()

,comp_id,comp_full_name,label_name,src_tags,one_src_tags,tags_list
812703,15907682253113655910,北京帕伯利森公关顾问有限公司,文化创意产业,文化创意产业-文娱传媒-文化与传媒-营销策略,文化创意产业-文娱传媒-文化与传媒-营销策略,"{文娱传媒, 营销策略, 文化创意产业, 文化与传媒}"
812704,15907682253113655910,北京帕伯利森公关顾问有限公司,文娱传媒,文化创意产业-文娱传媒-文化与传媒-营销策略,文化创意产业-文娱传媒-文化与传媒-营销策略,"{文娱传媒, 营销策略, 文化创意产业, 文化与传媒}"
812705,15907682253113655910,北京帕伯利森公关顾问有限公司,文化与传媒,文化创意产业-文娱传媒-文化与传媒-营销策略,文化创意产业-文娱传媒-文化与传媒-营销策略,"{文娱传媒, 营销策略, 文化创意产业, 文化与传媒}"
812706,15907682253113655910,北京帕伯利森公关顾问有限公司,营销策略,文化创意产业-文娱传媒-文化与传媒-营销策略,文化创意产业-文娱传媒-文化与传媒-营销策略,"{文娱传媒, 营销策略, 文化创意产业, 文化与传媒}"
812710,4925886916505930450,大连金普新区商品现货合约交易中心有限公司,大数据产业,大数据产业-数据处理,大数据产业-数据处理,"{数据处理, 大数据产业}"


In [17]:
ctag_filtered_count = data_ctag_filtered.groupby("label_name").agg({"comp_full_name": "count"}).sort_values(by="comp_full_name", ascending=False).reset_index()

### 根据公司名字过滤公司简介，并作合并、分词

In [26]:
data_intro_filtered = data_intro[(data_intro.comp_id.isin(data_ctag_filtered.comp_id)) & (data_intro.classify_id != 4)][["comp_id", "comp_full_name", "intro"]].copy()

In [27]:
data_intro_filtered.head(1)

,comp_id,comp_full_name,intro
14,15907682253113655910,北京帕伯利森公关顾问有限公司,帕伯利森公司是国内专业的企业第三方公关营销服务机构，为企业提供公司品牌战略级或产品级的公关营...


In [29]:
data_intro_filtered_merged = data_intro_filtered.groupby("comp_id").agg({"comp_full_name": max, "intro": lambda x: "。".join(x)})

In [30]:
len(data_intro_filtered_merged), len(set(data_intro_filtered.comp_id)), len(set(data_ctag_filtered.comp_id))

(100965, 100965, 100965)

In [ ]:
data_intro_filtered_merged["words"] = data_intro_filtered_merged.intro.apply(lambda x: jieba.lcut(x.strip()))

In [32]:
data_intro_filtered_merged.head(2)

,comp_full_name,intro,words
comp_id,,,
10000030533280313266,丽江童话假日旅行社有限公司,童话假日旅行是丽江童话假日旅行社所经营的滇西地区影响力比较大的纯玩品质旅行品牌，丽江童话...,"[童话, 假日, 旅行, 是, 丽江, 童话, 假日, 旅行社, 所, 经营, 的, 滇西,..."
10000039786266589368,梦幻影音（北京）文化传播有限公司,梦幻影音（北京）文化传播有限公司成立于2015年12月4日，位于北京市朝阳区，是一家影视创意...,"[梦幻, 影音, （, 北京, ）, 文化, 传播, 有限公司, 成立, 于, 2015, ..."


#### 去除停用词（标点）以及单字

In [38]:
stopwords = open("../Data/Input/text_similarity/stopwords.txt", "r").read().split("\n")
stopwords[0] = "，"
stopwords = set(stopwords)

In [45]:
data_intro_filtered_merged.words = data_intro_filtered_merged.words.apply(lambda x: list(filter(lambda w: len(w) > 1 and w not in stopwords, x)))

In [59]:
data_intro_filtered_merged.head(1)

,comp_full_name,intro,words
comp_id,,,
10000030533280313266,丽江童话假日旅行社有限公司,童话假日旅行是丽江童话假日旅行社所经营的滇西地区影响力比较大的纯玩品质旅行品牌，丽江童话...,"[童话, 假日, 旅行, 丽江, 童话, 假日, 旅行社, 经营, 滇西, 地区, 影响力,..."


In [5]:
model = gensim.models.KeyedVectors.load_word2vec_format("../Data/sgns.baidubaike.bigram-char")

In [118]:
pickle.dump(data_ctag_filtered, open("../Data/Input/Text_CNN/data_ctag_filtered.pkl", "wb"))
pickle.dump(data_intro_filtered_merged, open("../Data/Input/Text_CNN/data_intro_filtered_merged.pkl", "wb"))

In [2]:
data_ctag_filtered = pickle.load(open("../Data/Input/Text_CNN/data_ctag_filtered.pkl", "rb"))
data_intro_filtered_merged = pickle.load(open("../Data/Input/Text_CNN/data_intro_filtered_merged.pkl", "rb"))

In [4]:
words_raw = data_intro_filtered_merged.words.tolist()
words_list = [x for y in words_raw for x in y]
words_count = Counter(words_list)

#### fasttext词向量

In [10]:
fast_text_model = gensim.models.FastText(words_raw, min_count=5, size=300)

In [ ]:
fast_text_model.wv.most_similar(["科技", "金融"], topn=100)

#### fastest分类测试（只取底级标签）

In [20]:
label_list = ctag_filtered_count[ctag_filtered_count.comp_full_name >= 500].label_name.tolist()

In [27]:
label_data_to_use = data_ctag_filtered[(data_ctag_filtered.label_name.isin(label_list)) \
                                       & (data_ctag_filtered[["label_name", "one_src_tags"]].apply(lambda x: x[0] == x[1].split("-")[-1] , axis=1))]

In [38]:
all_data = label_data_to_use.merge(data_intro_filtered_merged.reset_index(), how="left", left_on="comp_id", right_on="comp_id")[["comp_id", "label_name", "words"]]

In [40]:
all_data.label_name = all_data.label_name.apply(lambda x: "__label__" + x)

In [46]:
all_data["data_to_file"] = all_data[["label_name", "words"]].apply(lambda x: " ".join([x[0], " ".join(x[1])]), axis=1)

In [48]:
all_data.head()

,comp_id,label_name,words,data_to_file
0,4925886916505930450,__label__数据处理,"[大连, 金普, 新区, 商品, 现货, 合约, 交易中心, 有限公司, 一家, 经营, 贵...",__label__数据处理 大连 金普 新区 商品 现货 合约 交易中心 有限公司 一家 经...
1,9780838354877475270,__label__LED,"[东莞市, 真泽, 电子, 有限公司, 创立, 2010, 位于, 东莞市, 谢岗镇, 大龙...",__label__LED 东莞市 真泽 电子 有限公司 创立 2010 位于 东莞市 谢岗镇...
2,17881009136713614790,__label__物流服务,"[公司, 国内, 知名, 快递, 物流, 企业, 韵达, 发展, 理念, 引领, 公司, 科...",__label__物流服务 公司 国内 知名 快递 物流 企业 韵达 发展 理念 引领 公司...
3,7849064920078805900,__label__文学,"[品书, 传播, 机构, 国内, 第一家, 图书, 专业, 推广, 机构, 致力于, 阅读,...",__label__文学 品书 传播 机构 国内 第一家 图书 专业 推广 机构 致力于 阅读...
4,424408944238282740,__label__广告服务,"[广东, 秒趣, 科技, 有限公司, 一家, 专注, 互联网, 服务, 高新技术, 企业, ...",__label__广告服务 广东 秒趣 科技 有限公司 一家 专注 互联网 服务 高新技术 ...


In [51]:
train_data, test_data = train_test_split(all_data, train_size=0.8)

/home/zijun.wu/packages/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [53]:
train_file = open("../Data/Input/Text_CNN/fasttext_train", "w")
train_file.write("\n".join(train_data.data_to_file.tolist()))
train_file.close()

38285577

In [55]:
test_file = open("../Data/Input/Text_CNN/fasttext_test", "w")
test_file.write("\n".join(test_data.data_to_file.tolist()))
test_file.close()

In [56]:
classifier = fasttext.supervised("../Data/Input/Text_CNN/fasttext_train", "fasttext.model", dim=300, loss="hs")

In [59]:
result = classifier.test("../Data/Input/Text_CNN/fasttext_test")

In [70]:
index_num = 0
test_text = " ".join(test_data.iloc[0].data_to_file.split(" ")[1:])

In [74]:
test_df = all_data[["label_name", "words"]].copy()

In [87]:
test_df["top_result"] = classifier.predict(test_df.words.apply(lambda x: " ".join(x)).tolist(),k=3)
test_df["real_label"] = test_df.label_name.apply(lambda x: x.strip("__label__"))
test_df["pred_result"] = test_df[["real_label", "top_result"]].apply(lambda x: x[0] in x[1], axis=1)

In [88]:
sum(test_df.pred_result)/len(test_df)

0.8080937521398457

In [89]:
test_df

,label_name,words,top_result,real_label,pred_result
0,__label__数据处理,"[大连, 金普, 新区, 商品, 现货, 合约, 交易中心, 有限公司, 一家, 经营, 贵...","[社会经济咨询, 汽车消费, 一般性求职招聘服务]",数据处理,False
1,__label__LED,"[东莞市, 真泽, 电子, 有限公司, 创立, 2010, 位于, 东莞市, 谢岗镇, 大龙...","[LED, 铝材料, 广告设计]",LED,True
2,__label__物流服务,"[公司, 国内, 知名, 快递, 物流, 企业, 韵达, 发展, 理念, 引领, 公司, 科...","[物流服务, 供应链管理服务, 商务代理代办服务]",物流服务,True
3,__label__文学,"[品书, 传播, 机构, 国内, 第一家, 图书, 专业, 推广, 机构, 致力于, 阅读,...","[文学, 人力资源培训服务, 市场调查]",文学,True
4,__label__广告服务,"[广东, 秒趣, 科技, 有限公司, 一家, 专注, 互联网, 服务, 高新技术, 企业, ...","[社会经济咨询, 广告设计, 数据分析]",广告服务,False
5,__label__建筑设计,"[南京, 北泰, 玻璃制品, 有限公司, 一家, 专业, 致力于, 节能, 环保, 建筑, ...","[铝材料, 建筑设计, 新型材料产业]",建筑设计,True
6,__label__影视,"[剧淘是, 一款, 看边, 影视娱乐, 粉丝, 电商, 社区, APP, 国内, 领先, 影...","[影视, VR产业, 数据服务]",影视,True
7,__label__工艺品,"[中艺合, 文化, 致力于, 打造, 艺术, 珍品, 鉴赏, 品牌, 领略到, 中国书画, ...","[工艺品, 绘画, 文化创意产业]",工艺品,True
8,__label__多媒体设计,"[京纪, 中达, 北京, 科技, 有限公司, 一家, 致力于, 多媒体, 投影, 显示, 系...","[多媒体设计, 商务服务, 办公服务]",多媒体设计,True
9,__label__铜,"[中铝, 东南, 铜业, 有限公司, 简称, 东南, 铜业, 中国, 铝业, 公司, 福建省...","[物流服务, 铜, 一般性求职招聘服务]",铜,True


#### 思路记录
可以选取大语料（优质、不优质都包含）训练词向量，然后任意两组词可以计算距离，那么等同于可以计算任意两个公司的距离